# Run Vcontact2 with the prophage genomes
***

Generate the input files 

In [ ]:
# Step one : generating the fasta file with the phages proteins and the gene_to_genome file

from os import listdir, mkdir, rename, system
import os


path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70"
path_phylogeny="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"



with open(f"{path_phylogeny}/phageboost_70_Vcontact2.fasta","w") as outfile :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.csv","w") as outfile2:
        outfile2.write("protein_id,contig_id,keywords\n")
        for rep in os.listdir(path_phageboost) :
            for strain in os.listdir(f"{path_phageboost}/{rep}") :
                for file in os.listdir(f"{path_phageboost}/{rep}/{strain}") :
                    if file[-6:]==".fasta":
                        prophage_name=file.split(".fasta")[0]
                        prot_file=open(f"{path_phageboost}/{rep}/{strain}/{prophage_name.split('__')[-1]}.multi.faa").read()
                        outfile.write(f"{prot_file}\n")
                        prot_file_id=[seq.split("\n")[0] for seq in prot_file.split(">")]
                        for index_seq, seq_id in enumerate(prot_file_id):
                            if seq_id:
                                outfile2.write(f"{seq_id},{prophage_name},None_provided\n")
                

#!/bin/bash
#BATCH --job-name=writting_Vcon
#SBATCH --partition=short 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=10
#SBATCH --mem=75gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=writting_Vcon%j.log 

module restore la_base
                
                
python3 /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/script_file/py_files/generate_vcontact2_files.py

Merge the previously created input files with the Millard data

In [ ]:
from os import listdir, mkdir, rename, system
import os


path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70"
path_phylogeny="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"
path_77="/home/conchae/77_strains_phage_project"
path_millard = "/home/conchae/databases/Millard_sept_2022"

gene2genome_m = "\n".join(open(f"{path_millard}/1Sep2022_vConTACT2_gene_to_genome.csv").read().split("\n")[1:])
gene_seq_m = open(f"{path_millard}/1Sep2022_vConTACT2_proteins.faa").read()

PPT_gene2genome = open(f"{path_phylogeny}/phageboost_70_gene_to_genome.csv").read()
PPT_gene_seq = open(f"{path_phylogeny}/phageboost_70_Vcontact2.fasta").read()

with open(f"{path_phylogeny}/phageboost_70_Vcontact2.Millard.fasta","w") as outfile :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.Millard.csv","w") as outfile2:
        outfile.write(f"{PPT_gene_seq}\n{gene_seq_m}")
        outfile2.write(f"{PPT_gene2genome}\n{gene2genome_m}")
        
        
        
# ************************************************************************************************************************************************       
#!/bin/bash
#BATCH --job-name=writting_Vcon
#SBATCH --partition=short 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=10
#SBATCH --mem=35gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=writting_Vcon%j.log 

module restore la_base
                
python3 /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/script_file/py_files/generate_vcontact2_files.Millard.py

In [ ]:
# Step two : The actual command line :

#!/bin/bash
#BATCH --job-name=vcontact2_phageboost
#SBATCH --partition=long 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60
#SBATCH --mem=250gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=vcontact2_phageboost%j.log 

module restore la_base
source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate vcontact2


vcontact  --raw-proteins /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_Vcontact2.Millard.fasta --rel-mode 'Diamond' --proteins-fp /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_gene_to_genome.Millard.csv --db 'None'  --pcs-mode MCL --vcs-mode ClusterONE --c1-bin /home/conchae/software/cluster_one-1.0.jar --output-dir  /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_06102022 -t 60



--blast-fp /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_06102022/phageboost_70_Vcontact2.Millard.self-diamond.tab 

In [ ]:
from os import listdir, mkdir, rename, system
import os

path_77="/home/conchae/77_strains_phage_project"
path_77_seq="/home/conchae/77_strains_phage_project/phages_sequences_corrected"

vcontact2_results=open(path_77+"/vConTACT2_77_phages_V5/genome_by_genome_overview.csv").read().split("\n")
reference={}
our_phages=[]

for phage in os.listdir(path_77_seq):
    our_phages.append(phage)
    
for n in range (1,len(vcontact2_results)-1):
    phage=vcontact2_results[n].split(",")[1]
    if phage not in our_phages:
        V_C=vcontact2_results[n].split(",")[5]
        order=vcontact2_results[n].split(",")[2]
        family=vcontact2_results[n].split(",")[3]
        genus=vcontact2_results[n].split(",")[4]
        score=vcontact2_results[n].split(",")[-1]
        if V_C !="" and V_C not in reference and float(score) > 0.9 :
            a={"Order":order}
            b={"Family":family}
            c={"Genus":genus}
            a.update(b)
            a.update(c)
            reference[V_C]=a

with open(path_77+"/Vcontact_77_results.tsv","w") as outfile:
    for n in range(1,len(vcontact2_results)-1):
        phage=vcontact2_results[n].split(",")[1]
        if phage in our_phages:
            V_C=vcontact2_results[n].split(",")[5]
            score=vcontact2_results[n].split(",")[-1]
            try :
                if float(score) > 0.9 and V_C in reference:
                    outfile.write(phage+"\t"+reference[V_C]["Order"]+"\t"+reference[V_C]["Family"]+"\t"+reference[V_C]["Genus"]+"\t"+V_C+"\t"+score+"\n")
                elif float(score) > 0.7 and V_C in reference :
                    outfile.write(phage+"\t"+reference[V_C]["Order"]+"\t"+reference[V_C]["Family"]+"\t"+V_C+"\t"+score+"\n")
                elif V_C =="67_0":
                    outfile.write(phage+"\t Caudovirales \t Siphoviridae \n")
                elif V_C not in reference :
                    print(phage, V_C)
                    outfile.write(vcontact2_results[n]+"\n")
                else:
                    print(phage)
            except ValueError:
                outfile.write(vcontact2_results[n]+"\n")